In [1]:
# base setup
from z3 import *

# 1. Выражения, сорта и декларации

Выражения, сорта и декларации в Z3 доступны в качестве абстрактных синтаксических деревьев (ASTs).

**Задача №1**
Представим мы проектируем DSL, в котором разрешено объявлять пользовательсик функции. Каждая функция должна иметь строгую сигнатуру: заранее известное число аргументов и типы этих аргументов.  
Передается выражения, к примеру `f(x, y)` или `f(x, 2.5)` - после чего необходимо проверить корректность применения функции, а именно действительно ли `.decl()` из `f(x, y)` совпадает с самой функцией `f` через `eq()`.

P.S. приложение - вызов функции или использование оператора с аргументами.

In [6]:
from z3 import *

# Объявление переменных
x, y = Ints('x y')

# Объявление функции f: (Int, Real) -> Bool
f = Function('f', IntSort(), RealSort(), BoolSort())

# Инициализация приложения
app = f(x, ToReal(y))


# РЕШЕНИЕ
print("Cовпадает ли декларация:", eq(f, app.decl()))

Cовпадает ли декларация: True


**Задача №2**
Необходимо создать утверждение с квантором:
$$∀x ∈ ℤ, ∃y ∈ ℤ. f(x, ToReal(y)) == True$$

P.S. кванторы это - конструкции помогающие выражать обобщения
     утверждение это - что-то что можно проверить на истинность

In [9]:
# РЕШЕНИЕ
quantified = ForAll([x], Exists([y], f(x, ToReal(y)) == True))
print("Формула с кванторами:", quantified)

# Проверка выполнимости
s = Solver()
s.add(Not(quantified))
if s.check() == sat:
    print("Формула опровергается при:", s.model())

Формула с кванторами: ForAll(x, Exists(y, f(x, ToReal(y)) == True))
Формула опровергается при: [f = [else -> False]]


# 2. Массивы

**Задача №1** Проверка согласованности истории логов. Здесь мы можем представить логи в качества массивов. Каждый элемент массива `log[i]`- это результат выполнения команды с номером `i`. Необходимо определить механику перезаписи лога и противоречит ли оно логике ниже:   
```
Если две команды `i` и `j` записывают разные значения в один и тот же слот и `i < j`, то после команды `j` в этом слоте должно быть второе значение.
```

In [16]:
# Абстракция слота
slot = Int('slot')

# Команды
i, j = Ints('i j')
v1, v2 = Ints('v1 v2')

# Исходный массив с логами
log = Array('log', IntSort(), IntSort())

# После команды i
log_i = Store(log, slot, v1)

# После команды j
log_j = Store(log_i, slot, v2)

# Проверка утверждения что после j в этом слоте должно быть v2
assertion = Implies(
    And(i < j, v1 != v2),
    Select(log_j, slot) == v2
)

# Проверка доказуемости
prove(assertion)  # должно вывести `proved`

proved


**Задача №2** Данная задача является продолжением второй, только теперь необходимо доказать что: 

```
Если i != j и Select(Store(log, i, v1), j) != Select(Store(log, j, v2), j) → значит был конфликт.
```

In [18]:
# Переменные
i, j, v1, v2 = Ints('i j v1 v2')
log = Array('log', IntSort(), IntSort())

# Два разных состояния массива
log_i = Store(log, i, v1)  # запись v1 по индексу i
log_j = Store(log, j, v2)  # запись v2 по индексу j

# Значения при чтении по j индексу
val_from_i = Select(log_i, j)
val_from_j = Select(log_j, j)

# Формулировка утверждения
conflict_detected = Implies(
    And(i != j, val_from_i != val_from_j),
    True  # если условие верно — конфликт точно есть
)

# Проверка на логическое тождество
prove(conflict_detected)

proved


# 3. Кванторы

**Задача №1** Представим что мы моделируем систему типов с подтипизацией и ковариантностью массивов.   
Нужно проверить: при любых x и y, если y является подтипом x, то array_of(y) можно использовать там, где требуется array_of(x)

In [21]:
# РЕШЕНИЕ
# Объектный тип
Type = DeclareSort('Type')

# Функции
subtype = Function('subtype', Type, Type, BoolSort())
array_of = Function('array_of', Type, Type)

# Переменные
x, y = Consts('x y', Type)

# Формирование тверждения
covariance = ForAll([x, y], Implies(subtype(y, x), subtype(array_of(y), array_of(x))))

s = Solver()
s.add(Not(covariance))  # .add на поиск контрпримера

# Основная проверка
if s.check() == unsat:
    print("Свойство ковариантности доказано")
else:
    print("Нарушение ковариантности, контрпример:")
    print(s.model())

Нарушение ковариантности, контрпример:
[subtype = [(Type!val!2, Type!val!3) -> False, else -> True],
 array_of = [Type!val!1 -> Type!val!3, else -> Type!val!2]]


# 4. Оптимизация

# 5. Множественные солверы